# Customizing visual appearance



HoloViews elements like the `Scatter` points illustrated in the [Introduction](1-Introduction.ipynb) contain two types of information:

- **Your data**, in as close to its original form as possible, so that it can be analyzed and accessed as you see fit.
- **Metadata specifying what your data *is***, which allows HoloViews to construct a visual representation for it.

What elements do *not* contain is:

- The endless details that one might want to tweak about the visual representation, such as line widths, colors, fonts, and spacing.

HoloViews is designed to let you work naturally with the meaningful features of your data, while making it simple to adjust the display details separately using the Options system.  Among many other benefits, this separation of *content* from *presentation* simplifies your data analysis workflow, and makes it independent of any particular plotting backend.

## Visualizing neural spike trains

To illustrate how the options system works, we will use a dataset containing ["spike"](https://en.wikipedia.org/wiki/Action_potential) (neural firing) events extracted from the recorded electrical activity of a [neuron](https://en.wikipedia.org/wiki/Neuron). We will be visualizing the first trial of this [publicly accessible neural recording](http://www.neuralsignal.org/data/04/nsa2004.4/433l019). First, we import pandas and holoviews and load our data:

In [ ]:
import pandas as pd
import holoviews as hv
from holoviews import opts
spike_train = pd.read_csv('../assets/spike_train.csv.gz')
spike_train.head(n=3)

This dataset contains the spike times (in milliseconds) for each detected spike event in this five-second recording, along with a spiking frequency (in Hertz, averaging over a rolling 200 millisecond window). We will now declare ``Curve`` and ``Spike`` elements using this data and combine them into a ``Layout``:

In [ ]:
curve = hv.Curve(spike_train, 'milliseconds', 'Hertz', group='Firing Rate')
spikes = hv.Spikes(spike_train.sample(300), kdims='milliseconds', vdims=[], group='Spike Train')
curve + spikes

Notice that the representation for this object is purely textual; so far we have not yet loaded any plotting system for HoloViews, and so all you can see is a description of the data stored in the elements. 

To be able to see a visual representation and adjust its appearance, we'll need to load a plotting system, and here let's load two so they can be compared:

In [ ]:
hv.extension('bokeh', 'matplotlib')

Even though we can happily create, analyze, and manipulate HoloViews objects without using any plotting backend, this line is normally executed just after importing HoloViews so that objects can have a rich graphical representation rather than the very-limited textual representation shown above. Putting 'bokeh' first in this list makes visualizations default to using [Bokeh](http://bokeh.pydata.org), but including [matplotlib](http://matplotlib.org) as well means that backend can be selected for any particular plot as shown below.

# Default appearance

With the extension loaded, let's look at the default appearance as rendered with Bokeh:

In [ ]:
curve + spikes

As you can see, we can immediately appreciate more about this dataset than we could from the textual representation.  The curve plot, in particular, conveys clearly that the firing rate varies quite a bit over this 5-second interval.  However, the spikes plot is much more difficult to interpret, because the plot is nearly solid black even though we already downsampled from 700 spikes to 300 spikes when we declared the element.  

One thing we can do is enable one of Bokeh's zoom tools and zoom in until individual spikes are clearly visible.  Even then, though, it's difficult to relate the spiking and firing-rate representations to each other.  Maybe we can do better by adjusting the display options away from their default settings?

## Customization

Let's see what we can achieve when we do decide to customize the appearance:

In [ ]:
curve  = hv.Curve( spike_train, 'milliseconds', 'Hertz')
spikes = hv.Spikes(spike_train, 'milliseconds', [])
layout = (curve + spikes).cols(1)
layout.opts(
    opts.Curve( height=150, width=900, xaxis=None, tools=['hover'], color='red', line_width=1.5),
    opts.Spikes(height=150, width=900, yaxis=None, color='grey', line_width=0.25))

Much better! It's the same underlying data, but now we can clearly see both the individual spike events and how they affect the moving average.  You can also see how the moving average trails the actual spiking, due to how the window function was defined.

A detailed breakdown of this exact customization is given in the [User Guide](../user_guide/03-Customizing_Plots.ipynb), but we can use this example to understand a number of important concepts:

* The layout container has a ``cols`` method to specify the number of columns in the layout.
* The option system is based around keyword settings supplied to the `.opts` method.
* You can style the components of composite objects in a single `.opts` method call. 
* The `opts` utility supplies both early *validation* of keywords as well as *tab-completion* in IPython.
* HoloViews also supports a set of IPython specific *magics* for setting options. Although these magics can be convenient in the notebook environment, they are not standard Python syntax.

The corresponding [User Guide](../user_guide/03-Customizing_Plots.ipynb) entry explains the keywords used in detail, but a quick summary is that when you tab-complete using the `opts` utility, you are completing across two fundamental types of options called ***plot options*** and ***style options***. HoloViews will manage this distinction for you but it is worth noting that the `color` and `line_width` keywords passed to the `Curve` and `Spikes` option objects correspond to the color and line widths of the corresponding [Bokeh glyphs](http://bokeh.pydata.org/en/latest/docs/user_guide/plotting.html).

As you can see, these tools allow significant customization of how our elements appear. HoloViews offers many other tools for setting options either locally or globally, including the aforementioned IPython specific magics, are described in the [User Guide](../user_guide/03-Customizing_Plots.ipynb). This user guide includes a recommendation on how you can customize sophisticated, composite visualizations in a consistent way.

# Switching to matplotlib

Now let's switch our backend to [matplotlib](http://matplotlib.org/) to show the same elements in `layout` as rendered with different customizations, in a different output format (SVG), with a completely different plotting library:

In [ ]:
hv.output(backend='matplotlib', fig='svg')
layout.opts(
    opts.Curve(aspect=6, xaxis=None, show_grid=False, color='blue', linewidth=2, linestyle='dashed'),
    opts.Spikes(aspect=6, yaxis='bare', color='red', linewidth=0.25),
    opts.Layout(sublabel_format='', vspace=0.1, fig_size=200))

Here we use the same tools with a different plotting extension. Naturally, a few changes needed to be made:

* A few of the options are different because of differences in how the plotting backends work. For instance, matplotlib uses ``aspect`` instead of setting ``width`` and ``height``. In some cases, but not all, HoloViews can smooth over such differences in the *plotting* options to make it simpler to switch backends.
* The Bokeh hover tool is not supported by the matplotlib backend, as you might expect, nor are there any other interactive controls.
* Some options have different names; for instance, the Bokeh ``line_width`` option is called ``linewidth`` in matplotlib. These options are directly inherited from the API of the plotting library and are the style options.
* Containers like `Layout`s also have options to control the arrangement of its components. Here we adjust the gap betwen the plots using ``vspace``.

Note that you can even write options that work across multiple backends, as HoloViews will ignore keywords that are not applicable to the current backend (as long as they are valid for *some* loaded backend). See the [User Guide](../user_guide/03-Customizing_Plots.ipynb) for more details.

## Persistent styles

Let's switch back to the default (Bokeh) plotting extension for this notebook and apply the ``.select`` operation illustrated in the Introduction, to the ``spikes`` object we made earlier:

In [ ]:
hv.output(backend='bokeh')
spikes.select(milliseconds=(2000,4000))

Note how HoloViews remembered the Bokeh-specific styles we previously applied to the `spikes` object! This feature allows us to style objects once and then keep that styling as we work, without having to repeat the styles every time we work with that object. You can learn more about the output line magic and the exact semantics of the opts magic in the [User Guide](../user_guide/03-Customizing_Plots.ipynb).

## Setting axis labels

If you look closely, the example above might worry you. First we defined our ``Spikes`` element with  ``kdims=['milliseconds']``, which we then used as a keyword argument in ``select`` above. This is also the string used as the axis label. Does this mean we are limited to Python literals for axis labels, if we want to use the corresponding dimension with ``select``?

Luckily, there is no limitation involved.  Dimensions specified as strings are often convenient, but behind the scenes, HoloViews always uses a much richer ``Dimensions`` object which you can pass to the ``kdims`` and ``vdims`` explicitly (see the [User Guide](../user_guide/01-Annotating_Data.ipynb) for more information). One of the things each ``Dimension`` object supports is a long, descriptive ``label``,  which complements the short programmer-friendly name.

We can set the dimension labels on our existing ``spikes`` object as follows:

In [ ]:
spikes = spikes.redim.label(milliseconds='Time in milliseconds (10⁻³ seconds)')
curve  = curve.redim.label(Hertz='Frequency (Hz)')
(curve + spikes).select(milliseconds=(2000,4000)).cols(1)

As you can see, we can set long descriptive labels on our dimensions (including unicode) while still making use of the short dimension name in methods such as ``select``.  

Now that you know how to set up and customize basic visualizations, the next [Getting-Started sections](./3-Tabular_Datasets.ipynb) show how to work with various common types of data in HoloViews.